In [89]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta



In [90]:
def calculate_dates(original_date):
    now = datetime.now()
    delta1 = datetime(now.year, original_date.month, original_date.day)
    delta2 = datetime(now.year+1, original_date.month, original_date.day)
    
    return ((delta1 if delta1 > now else delta2) - now).days + 1



In [91]:
calculate_dates(bdays['birthday'][5])

340

In [92]:
bdays = pd.read_csv("src/birthdays.csv", 
                usecols=["name","email","birthday"],
                parse_dates=["birthday"])


bdays['birthday_year'] = bdays['birthday'].dt.year
bdays['birthday_date'] = bdays['birthday'].dt.strftime('%m-%d')

bdays['birthday_date_7'] = bdays['birthday'] - timedelta(days=7)
bdays['birthday_date_1'] = bdays['birthday'] - timedelta(days=1)
bdays['birthday_date_7'] = bdays['birthday_date_7'].dt.strftime('%m-%d')
bdays['birthday_date_1'] = bdays['birthday_date_1'].dt.strftime('%m-%d')

bdays['days_until_bday'] = bdays.apply(lambda bdays: calculate_dates(bdays['birthday']), axis=1)

In [93]:
reminders = pd.read_csv("src/birthdays.csv", 
                usecols=["name","email","frequency"])

reminders["7_before"] = reminders["frequency"].str.contains('1 week ervoor').astype(int)
reminders["1_before"] = reminders["frequency"].str.contains('1 dag ervoor').astype(int)
reminders["0_before"] = reminders["frequency"].str.contains('Op de dag').astype(int)
reminders = reminders.drop(columns=["frequency"])

In [94]:
current_date = datetime.today().strftime('%m-%d')
current_plus_1 = (datetime.today() + timedelta(days=1)).strftime('%m-%d')
current_plus_7 = (datetime.today() + timedelta(days=7)).strftime('%m-%d')

print(f"Date: {current_date}")
print(f"Date + 1: {current_plus_1}")
print(f"Date + 7: {current_plus_7}")

Date: 01-24
Date + 1: 01-25
Date + 7: 01-31


In [95]:
bdays

,name,email,birthday,birthday_year,birthday_date,birthday_date_7,birthday_date_1,days_until_bday
0,Tristan,tristan@test.nl,2023-08-18,2023,08-18,08-11,08-17,207
1,Bo Donkersloot,bo@test.nl,1997-08-30,1997,08-30,08-23,08-29,219
2,Marijn,marijn2huis@gmail.com,1997-08-16,1997,08-16,08-09,08-15,205
3,Kwinten,kwinten@test.nl,1997-08-18,1997,08-18,08-11,08-17,207
4,Pa,daddy@test.nl,1997-11-04,1997,11-04,10-28,11-03,285
5,Pim,pimduif@test.nl,1996-12-29,1996,12-29,12-22,12-28,340
6,Don,donno@test.nl,1997-09-19,1997,09-19,09-12,09-18,239
7,Ronald,ronno@test.nl,1995-08-23,1995,08-23,08-16,08-22,212
8,Test bro1,testbro1@test.nl,1997-01-22,1997,01-22,01-15,01-21,364
9,Test bro2,testbro2@test.nl,1998-01-23,1998,01-23,01-16,01-22,365


In [96]:
reminders

,name,email,7_before,1_before,0_before
0,Tristan,tristan@test.nl,1,1,1
1,Bo Donkersloot,bo@test.nl,0,0,1
2,Marijn,marijn2huis@gmail.com,1,1,1
3,Kwinten,kwinten@test.nl,1,1,1
4,Pa,daddy@test.nl,0,0,1
5,Pim,pimduif@test.nl,1,0,1
6,Don,donno@test.nl,0,1,0
7,Ronald,ronno@test.nl,0,1,0
8,Test bro1,testbro1@test.nl,1,0,1
9,Test bro2,testbro2@test.nl,1,0,1


In [97]:
remind_0_df = bdays.loc[(bdays['birthday_date']==current_date)].reset_index(drop=True)
remind_1_df = bdays.loc[(bdays['birthday_date']==current_plus_1)].reset_index(drop=True)
remind_7_df = bdays.loc[(bdays['birthday_date']==current_plus_7)].reset_index(drop=True)


In [98]:
def get_names_string(df:pd.DataFrame):
    names_str = ', '.join([x for x in df['name'].values])
    if len(df['name'].values) >= 2:
        names_str = ' and'.join(names_str.rsplit(',', 1))
    return names_str

In [99]:
for index, row in reminders.iterrows():
    if row['7_before']:
        print(f"Send email to {row['name']}, {row['email']}, with 7 day reminder for {get_names_string(remind_7_df)}")
    if row['1_before']:
        print(f"Send email to {row['name']}, {row['email']}, with 1 day reminder for {get_names_string(remind_1_df)}")
    if row['0_before']:
        print(f"Send email to {row['name']}, {row['email']}, with 0 day reminder for {get_names_string(remind_0_df)}")
    

Send email to Tristan, tristan@test.nl, with 7 day reminder for 
Send email to Tristan, tristan@test.nl, with 1 day reminder for 
Send email to Tristan, tristan@test.nl, with 0 day reminder for 
Send email to Bo Donkersloot, bo@test.nl, with 0 day reminder for 
Send email to Marijn, marijn2huis@gmail.com, with 7 day reminder for 
Send email to Marijn, marijn2huis@gmail.com, with 1 day reminder for 
Send email to Marijn, marijn2huis@gmail.com, with 0 day reminder for 
Send email to Kwinten, kwinten@test.nl, with 7 day reminder for 
Send email to Kwinten, kwinten@test.nl, with 1 day reminder for 
Send email to Kwinten, kwinten@test.nl, with 0 day reminder for 
Send email to Pa, daddy@test.nl, with 0 day reminder for 
Send email to Pim, pimduif@test.nl, with 7 day reminder for 
Send email to Pim, pimduif@test.nl, with 0 day reminder for 
Send email to Don, donno@test.nl, with 1 day reminder for 
Send email to Ronald, ronno@test.nl, with 1 day reminder for 
Send email to Test bro1, testbro